In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns #visualization
import matplotlib.pyplot as plt #visualization
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py
import plotly.express as px

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **<span style="color:#7B68EE;">Flying, I thought I'd never learn that flying. I thought I'd spend my whole life trying. For flying is that ancient art of keeping one foot on the ground.</span>**

Flying - Songwriter: Chris de Burgh

<h1 style="background-color:#DC143C; font-family:'Brush Script MT',cursive;color:white;font-size:200%; text-align:center;border-radius: 50% 20% / 10% 40%">Activity Recognition system based on Multisensor data fusion (AReM) Data Set</h1>

Citation: F. Palumbo, C. Gallicchio, R. Pucci and A. Micheli, Human activity recognition using multisensor data fusion based on Reservoir Computing, Journal of Ambient Intelligence and Smart Environments, 2016, 8 (2), pp. 87-107.

This dataset represents a real-life benchmark in the area of Activity Recognition applications.

The classification tasks consist in predicting the activity performed by the user from time-series generated by a Wireless Sensor Network (WSN), according to the EvAAL competition technical annex.

Abstract: That dataset contains temporal data from a Wireless Sensor Network worn by an actor performing the activities: bending, cycling, lying down, sitting, standing, walking.

Attribute Information:

For each sequence, data is provided in comma separated value (csv) format.

Input RSS streams are provided in files named datasetID.csv, where ID is the progressive numeric sequence ID for each repetition of the activity performed.
In each file, each row corresponds to a time step measurement (in temporal order) and contains the following information:
avg_rss12, var_rss12, avg_rss13, var_rss13, avg_rss23, var_rss23
where avg and var are the mean and variance values over 250 ms of data, respectively.

Target data is provided as the containing folder name.

For each activity, we have the following parameters:

Frequency (Hz): 20

Clock (millisecond): 250

Total duration (seconds): 120

I removed those parameters with skiprows =4.

https://archive.ics.uci.edu/ml/datasets/Activity+Recognition+system+based+on+Multisensor+data+fusion+(AReM)

In [ ]:
#Code by Rawwar https://www.kaggle.com/rawwar/eda-pandas-profile-based-conclusions
df = pd.read_csv("../input/activity-recognition-data/lying/dataset4.csv",skiprows=4, delimiter=',')
df.head()

#AVG and VAR are the mean and variance values over 250 ms of data,

# **<span style="color:#7B68EE;">Lying, I thought I'd never keep from lying. I thought I'd lose it all by sighing. For lying is that ancient art of hiding words that will never be found.</span>**

Flying - Songwriter: Chris de Burgh

In [ ]:
df.isnull().sum()

#Codes by Rishabh Raj Shukla https://www.kaggle.com/rajshukla1102/beginnernotebook/notebook

In [ ]:
from pandas.plotting import scatter_matrix
from sklearn.decomposition import PCA
from sklearn import linear_model, decomposition
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df.plot(subplots=True, sharex=True ,figsize=(20,50))
plt.show()

In [ ]:
df = df.rename(columns={'# Columns: time':'time'})

In [ ]:
df.drop('time', axis=1).corrwith(df.time).plot(kind='bar', grid=True, figsize=(12, 8), color=['salmon'],
                                                   title="Correlation with Time")

In [ ]:
sc = StandardScaler()
scale_var = ['avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23']
df[scale_var] = sc.fit_transform(df[scale_var])

In [ ]:
X = df.drop('time', axis=1)
y = df.time

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
allAlgo = [('lr', LogisticRegression()),('knn', KNeighborsClassifier()),('dclf', DecisionTreeClassifier()),
          ('svm', SVC()),('nb', GaussianNB())]

In [ ]:
res = []
names = []
for name, algo in allAlgo:
    kfold = KFold(n_splits=10, random_state=None)
    cv_results = cross_val_score(algo, X_train, y_train, cv=kfold, scoring='accuracy')
    res.append(cv_results)
    names.append(name)
    print(name, cv_results.mean(), cv_results.std())

In [ ]:
def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print("Train Result:\n**********************************************")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")
        
    elif train==False:
        pred = clf.predict(X_test)
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        print("Test Result:\n************************************************")        
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

In [ ]:
lr = LogisticRegression(solver='saga',penalty='elasticnet',l1_ratio=0.6,max_iter=1000)
lr.fit(X_train, y_train)

print_score(lr, X_train, y_train, X_test, y_test, train=True)
print_score(lr, X_train, y_train, X_test, y_test, train=False)

#Applying PCA on Logistic Regression

In [ ]:
#Applying PCA on Logistic Regression

pca = PCA(n_components=5) 
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
expained_variance = pca.explained_variance_ratio_
classifier = LogisticRegression(random_state = 1)
classifier.fit(X_train_pca, y_train)
print_score(classifier, X_train_pca, y_train, X_test_pca, y_test, train=True)
print_score(classifier, X_train_pca, y_train, X_test_pca, y_test, train=False)

#Support Vector Machine

In [ ]:
sv = SVC(kernel='rbf',C=1)
sv.fit(X_train, y_train)
print_score(sv, X_train, y_train, X_test, y_test, train=True)
print_score(sv, X_train, y_train, X_test, y_test, train=False)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
print_score(knn, X_train, y_train, X_test, y_test, train=True)
print_score(knn, X_train, y_train, X_test, y_test, train=False)

In [ ]:
gn = GaussianNB()
gn.fit(X_train, y_train)
print_score(gn, X_train, y_train, X_test, y_test, train=True)
print_score(gn, X_train, y_train, X_test, y_test, train=False)

In [ ]:
#DecisionTreeClassifier

dc = DecisionTreeClassifier()
dc.fit(X_train, y_train)
print_score(dc, X_train, y_train, X_test, y_test, train=True)
print_score(dc, X_train, y_train, X_test, y_test, train=False)

In [ ]:
#Code by Olga Belitskaya https://www.kaggle.com/olgabelitskaya/sequential-data/comments
from IPython.display import display,HTML
c1,c2,f1,f2,fs1,fs2=\
'#eb3434','#eb3446','Akronim','Smokum',30,15
def dhtml(string,fontcolor=c1,font=f1,fontsize=fs1):
    display(HTML("""<style>
    @import 'https://fonts.googleapis.com/css?family="""\
    +font+"""&effect=3d-float';</style>
    <h1 class='font-effect-3d-float' style='font-family:"""+\
    font+"""; color:"""+fontcolor+"""; font-size:"""+\
    str(fontsize)+"""px;'>%s</h1>"""%string))
    
    
dhtml('Thanks Rishabh Raj Shukla for the script' )